In [ ]:
import cx_Oracle
import pandas as pd
import sqlalchemy
from sqlalchemy.exc import SQLAlchemyError

<h1>Pobieranie danych z bazy</h1>

In [ ]:
data = {}

dbtables = ['MEB_DGM', 'MEB_DMC', 'MEB_GROB', 'MEB_KO', 'MEB_KO_DGM', 'MEB_KO_RODZAJ', 'MEB_KO_STREFA', 'MEB_KS', 'ONI_CIRCUITS']
querys = [
#Tutaj występują duplikaty, ale jeszcze nie zdecydowaliśmy co z nimi zrobić
"""
SELECT * 
FROM Z3DMC.MEB_DGM
""",
#Tutaj występują duplikaty, ale jeszcze nie zdecydowaliśmy co z nimi zrobić, usunąłem tylko nulle i anomalię
#Po wykonaniu LEFT JOIN liczba rekordów się zwiększyła, trzeba usunąć duplikaty z DMC_CASTING ~ LO
"""
SELECT *
FROM Z3DMC.MEB_DMC
WHERE (DMC <> 'null') AND (DMC <> 'NIECZYTELNY')
""",
"""
SELECT *
FROM (
  SELECT
    t.*,
    ROW_NUMBER() OVER (PARTITION BY ID_DMC ORDER BY ID_MEB_GROB DESC) AS rn
  FROM
    Z3DMC.MEB_GROB t
) subquery
WHERE rn = 1
""",
"""
SELECT *
FROM (
  SELECT
    t.*,
    ROW_NUMBER() OVER (PARTITION BY ID_DMC ORDER BY ID_KO DESC) AS rn
  FROM
    Z3DMC.MEB_KO t
) subquery
WHERE rn = 1
""",
"""
SELECT *
FROM (
  SELECT
    t.*,
    ROW_NUMBER() OVER (PARTITION BY ID_DMC ORDER BY ID_KO DESC) AS rn
  FROM
    Z3DMC.MEB_KO_DGM t
  WHERE ID_DMC <> 0
) subquery
WHERE rn = 1
""",
#Tabela słownik, niczego nie usuwamy, wczytuję całą tabelę
"""
SELECT *
FROM Z3DMC.MEB_KO_RODZAJ
""",
#Tabela słownik, niczego nie usuwamy, wczytuję całą tabelę
"""
SELECT *
FROM Z3DMC.MEB_KO_STREFA
""",
"""
SELECT *
FROM (
  SELECT
    t.*,
    ROW_NUMBER() OVER (PARTITION BY ID_DMC ORDER BY ID_KS DESC) AS rn
  FROM
    Z3DMC.MEB_KS t
) subquery
WHERE rn = 1
""",
#Tabela ONI_CIRCUITS posiada wiele duplikatów, a powinno być ich do 28 per jeden ID_DMC, dlatego wrzucam całą tabelę i przegadamy co z tym zrobić.
#Poniżej zakomentowany kod usuwający wszystkie duplikaty.
"""
SELECT ID_DMC, CIRCUIT_NR, 
MAX(ASSIGMENT) AS ASSIGMENT, 
MAX(FLOW) AS FLOW, 
MAX(REF_TIME) AS REF_TIME,
MAX(SET_POINT) AS SET_POINT,
MAX(START_DELAY) AS START_DELAY,
MAX(TEMP) AS TEMP,
MAX(WORKING_MODE) AS WORKING_MODE,
MAX(TIMESTAMP) AS TIMESTAMP
FROM Z3DMC.ONI_CIRCUITS
GROUP BY ID_DMC, CIRCUIT_NR
ORDER BY ID_DMC
"""
#SELECT *
#FROM (
#  SELECT
#    t.*,
#    ROW_NUMBER() OVER (PARTITION BY ID_DMC ORDER BY ID_CIRCUIT DESC) AS rn
#  FROM
#    Z3DMC.ONI_CIRCUITS t
#) subquery
#WHERE rn = 1
]
username = 'dmcro'
password = 'ouZ9Rys7astDp4byjUvO'
dsn = 'vwpnwrud0000rac:1521/dmc.vwg'
dbhostname='vwpnwrud0000rac'
service_name='dmc.vwg'

In [ ]:
try:
    sqlalchemy_engine="oracle+cx_oracle://"+username+":"+password+"@"+dbhostname+"/?service_name="+service_name
    engine = sqlalchemy.create_engine(sqlalchemy_engine, arraysize=1000)
    for table, query in zip(dbtables, querys):
        data.update({table: pd.read_sql(query, engine)})
except SQLAlchemyError as e:
    print(e)

<h1>Usuwanie zbędnych kolumn z tabel</h1>

In [ ]:
print(list(data.keys()))
# Useless tables
# MEB_CONTAINERS, MEB_DMC_GRADE, MEB_KS_WZORC

In [ ]:
# ONI CIRCUITS
# print(data['ONI_CIRCUITS'].head())
data['ONI_CIRCUITS'].drop(columns=['timestamp'], inplace=True)
# print("\n ONI_CIRCUITS after drop of columns: \n")
# print(data['ONI_CIRCUITS'].head())

In [ ]:
# MEB_DGM
# print(data['MEB_DGM'].columns)
# print(data['MEB_DGM'].head(5))
# ('metal_level', 'metal_pressure') - We do not collect this data yet
data['MEB_DGM'].drop(columns=['timestamp', 'data_znakowania', 'data_odlania', 'metal_level', 'metal_pressure'], inplace=True)
# print("\n MEB_DGM after drop of columns: \n")
# print(data['MEB_DGM'].head(5))

In [ ]:
#MEB_DMC
# print(data['MEB_DMC'].columns)
# print(data['MEB_DMC'].head(5))
data['MEB_DMC'].drop(columns = ['timestamp', 'update_time','id_meb_containers', 'packed_time', 'first_packed_time', 'production_step'], inplace = True)
# print("\n MEB_DMC after drop of columns: \n")
# print(data['MEB_DMC'].head(5))

In [ ]:
#MEB_KO
# print(data['MEB_KO'].columns)
# print(data['MEB_KO'].head(5))
data['MEB_KO'].drop(columns = ['id_ko', 'data', 'timestamp', 'eks'], inplace = True)
# print("\n MEB_KO after drop of columns: \n")
# print(data['MEB_KO'].head(5))

In [ ]:
#MEB_KO_DGM
# print(data['MEB_KO_DGM'].columns)
# print(data['MEB_KO_DGM'].head(5))
data['MEB_KO_DGM'].drop(columns = ['id_ko','data_odlania', 'timestamp', 'operator'], inplace = True)
# print("\n MEB_KO_DGM after drop of columns: \n")
# print(data['MEB_KO_DGM'].head(5))

In [ ]:
# MEB_KS
# print(data['MEB_KS'].columns)
# print(data['MEB_KS'].head(5))
data['MEB_KS'].drop(columns = ['id_ks', 'nrgniazda', 'liczbawystapien', 'nrformy', 'data', 'timestamp', 'gradedmc_max','gradedmc_aktualny'], inplace = True)
# print("\n 'MEB_KS' after drop of columns: \n")
# print(data['MEB_KS'].head(5))

In [ ]:
#MEB_GROB 
# print(data['MEB_GROB'].columns)
# print(data['MEB_GROB'].head(5))
data['MEB_GROB'].drop(columns = ['shift_number', 'last_operation', 'timestamp', 'production_date', 'reworkrequested', 
                                'reworkdone', 'partcleaningisfinished', 'waitfortoolcheck', 'workingstep1', 'workingstep2', 'workingstep3', 'workingstep4', 'mms_ok'], inplace = True)
# print("\n 'MEB_GROB' after drop of columns: \n")
# print(data['MEB_GROB'].head(5))


<h1>Łączenie tabel</h1>

In [ ]:
# usuwanie znaków białych z DMC i DMC_CASTING
data['MEB_DMC'].dmc_casting = data['MEB_DMC'].dmc_casting.str.strip()
data['MEB_DGM'].dmc = data['MEB_DGM'].dmc.str.strip()

In [ ]:
# usuwanie z meb_dmc wierszy z 'WORKPIECE NIO' w kodzie DMC
print(data['MEB_DMC'].shape)
data['MEB_DMC'] = data['MEB_DMC'][~data['MEB_DMC']['dmc'].str.contains('WORKPIECE', case=False, na=False)]
print(data['MEB_DMC'].shape)

In [ ]:
final_table = data['MEB_DMC'].copy()

In [ ]:
# wybieranie produktu MEB+
data['MEB_DGM'] = data['MEB_DGM'][(data['MEB_DGM'].nr_dgm >= 8) & (data['MEB_DGM'].nr_dgm <= 10)]

In [ ]:
# łączę tabele MEB_KO i MEB_KO_DGM z tabelami MEB_KO_STREFA/RODZAJ

data['MEB_KO'] = data['MEB_KO'].merge(data['MEB_KO_STREFA'], left_on='nok_strefa', right_on='indeks', how='inner')
data['MEB_KO'].drop(columns=['indeks'], inplace=True)

data['MEB_KO'] = data['MEB_KO'].merge(data['MEB_KO_RODZAJ'], left_on='nok_rodzaj', right_on='indeks', how='inner')
data['MEB_KO'].drop(columns=['indeks'], inplace=True)

data['MEB_KO_DGM'] = data['MEB_KO_DGM'].merge(data['MEB_KO_STREFA'], left_on='nok_strefa', right_on='indeks', how='inner')
data['MEB_KO_DGM'].drop(columns=['indeks'], inplace=True)

data['MEB_KO_DGM'] = data['MEB_KO_DGM'].merge(data['MEB_KO_RODZAJ'], left_on='nok_rodzaj', right_on='indeks', how='inner')
data['MEB_KO_DGM'].drop(columns=['indeks'], inplace=True)


#data['MEB_KO'].head()
#data['MEB_KO_DGM'].head()

In [ ]:
# łączę tabelę MEB_DMC z tabelą MEB_KO
print(final_table.shape)
final_table = final_table.merge(data['MEB_KO'], on='id_dmc', how='left')
final_table.drop(columns=['rn'], inplace=True)
print(final_table.shape)

In [ ]:
# łączę tabelę MEB_DMC z tabelą MEB_GROB
print(final_table.shape)
final_table = final_table.merge(data['MEB_GROB'], on='id_dmc', how='left')
final_table.drop(columns=['rn'], inplace=True)
print(final_table.shape)

In [ ]:
# łączę tabelę MEB_DMC z tabelą MEB_KS
print(final_table.shape)
final_table = final_table.merge(data['MEB_KS'], on='id_dmc', how='left')
final_table.drop(columns=['rn'], inplace=True)
print(final_table.shape)

In [ ]:
# łączę tabelę MEB_DGM z tabelą MEB_KO_DGM
print(data['MEB_DGM'].shape)
data['MEB_DGM'] = data['MEB_DGM'].merge(data['MEB_KO_DGM'], left_on='id', right_on='id_dmc', how='left')
data['MEB_DGM'].drop(columns=['rn'], inplace=True)
print(data['MEB_DGM'].shape)

In [ ]:
data['MEB_DGM'][data['MEB_DGM'].id == data['MEB_DGM'].id_dmc]

In [ ]:
final_table.head()

In [ ]:
# przygotowywuję tabelę ONI_CIRCUITS do połączenia 

oni_circuits = data['ONI_CIRCUITS'].pivot(index='id_dmc', columns='circuit_nr', values=['assigment', 'flow', 'ref_time', 'set_point', 'start_delay', 'working_mode', 'temp'])
oni_circuits.columns = oni_circuits.columns.map('{0[0]}_{0[1]}'.format) 
oni_circuits.reset_index(inplace=True)


# łączę z tabelą MEB_DGM
print(f"Before meb_dgm to oni circuits {len(oni_circuits)}")
oni_circuits = oni_circuits.merge(data['MEB_DGM'], left_on='id_dmc', right_on='id', how='left')
print(f"After meb_dgm to oni circuits {len(oni_circuits)}")

In [ ]:
# Usunięcie nulli z oni circuits 'dmc'
oni_circuits = oni_circuits.dropna(subset=['dmc'])

In [ ]:
oni_circuits[oni_circuits.id_dmc_x == oni_circuits.id]

In [ ]:
oni_circuits.drop(columns=['id_dmc_y'], inplace=True)
oni_circuits.rename(columns={'id_dmc_x': 'id_dmc'}, inplace=True)

In [ ]:
print(list(oni_circuits.keys()))

In [ ]:
# łączę tabelę MEB_DMC z ONI_CIRCUITS
print(f"Before oni circuits to meb_dmc {len(final_table)}")
final_table = final_table.merge(oni_circuits, left_on='dmc_casting', right_on='dmc', how='left', suffixes=('_DMC', '_ONI'))
#data['MEB_DMC'].head(1)
print(f"after oni circuits to meb_dmc {len(final_table)}")

In [ ]:
duplicate_count = final_table['dmc_casting'].duplicated(keep=False).sum()
print(f"Number of rows with the same 'dmc_casting' value: {duplicate_count}")

In [ ]:
# nones_meb_dgm = oni_circuits['dmc'].isna().sum()
# print(nones_meb_dgm)
# oni_circuits = oni_circuits.dropna(subset=['dmc'])
# nones_meb_dgm = oni_circuits['dmc'].isna().sum()
# print(nones_meb_dgm)

In [ ]:
nones_meb_dmc = data['MEB_DMC']['dmc_casting'].isna().sum()
print(nones_meb_dmc)

In [ ]:
nones = final_table['dmc_casting'].isna().sum()
print(nones)

In [ ]:
duplicates = final_table[final_table['dmc_casting'].duplicated(keep=False)]
print("Duplicated rows:")
print(duplicates.head(10))

In [ ]:
print(list(final_table.keys()))

In [ ]:
duplicate_count_oni = final_table['dmc_DMC'].duplicated(keep=False).sum()
print(f"Number of rows with the same 'dmc' value: {duplicate_count_oni}")

In [ ]:
final_table.head()

<h1>Usuwanie anomalii</h1>

In [ ]:
print(list(data.keys()))

In [ ]:
meb_dgm_df = data['MEB_DGM']

In [ ]:
print(meb_dgm_df.head(5))

In [ ]:
print(list(meb_dgm_df.keys()))

In [ ]:
print(meb_dgm_df['nr_dgm'].unique())
print(len(meb_dgm_df))
print(type(meb_dgm_df['nr_dgm'][0]))
print(len(meb_dgm_df[meb_dgm_df['nr_dgm'] == 20]))

In [ ]:
print(final_table.head(4))

In [ ]:
for i in final_table.keys():
    print(i)

In [ ]:
result = meb_dgm_df.groupby(final_table['dmc_DMC'].str[:3]).size().reset_index(name='count_of_results')
result.columns = ['unique_starting_letters', 'count_of_results']
result = result.sort_values(by='unique_starting_letters')

print(result)

In [ ]:
meb_dgm_df[meb_dgm_df.dmc.str.len() < 21]

In [ ]:
data['MEB_DMC'][data['MEB_DMC'].dmc_casting.str.len() < 21]

In [ ]:
data['MEB_DMC'][data['MEB_DMC'].dmc.str.len() < 42]

In [ ]:
query = """select id_dmc, circuit_nr, 
max(assigment) as assigment, 
max(flow) as flow, 
max(ref_time) as ref_time,
max(set_point) as set_point,
max(start_delay) as start_delay,
max(temp) as temp,
max(working_mode) as working_mode,
max(timestamp) as timestamp
from z3dmc.oni_circuits
group by id_dmc, circuit_nr
order by id_dmc"""

query.upper()